In [1]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
import os
import warnings
warnings.filterwarnings('ignore')

df = pd.read_excel(r'C:\Users\serdar.yesildag\Desktop\VAM M&E\1Reports\FSMS\February 2023\Analysis\RAW Datasets\FSMSFeb23.xlsx')

In [2]:
MetaData = [col for col in df.columns if col[:2] == '@_']

In [3]:
Location = [col for col in df.columns if col[:5] == 'ADMIN' and col[-5:] != '3Name' and col[-5:] != '4Name' and col[-5:] != '5Name']

In [4]:
df['District'] = np.where(df['ADMIN2Name'] == 'CC101', 'Kailahun', df['ADMIN2Name'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC102', 'Kenema', df['District'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC103', 'Kono', df['District'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC104', 'Bombali', df['District'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC105', 'Falaba', df['District'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC106', 'Koinadugu', df['District'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC107', 'Tonkolili', df['District'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC108', 'Kambia', df['District'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC109', 'Karene', df['District'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC110', 'Port Loko', df['District'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC111', 'Bo', df['District'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC112', 'Bonthe', df['District'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC113', 'Moyamba', df['District'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC114', 'Pujehun', df['District'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC115', 'Western Area Rural', df['District'])
df['District'] = np.where(df['ADMIN2Name'] == 'CC116', 'Western Area Urban', df['District'])

In [5]:
df['ResGen'] = df['RESPSex'][df['RESPRelationHHH']==100]
df.ResGen.fillna(df.HHHSex, inplace=True)
df['HHHGender'] = df['ResGen'].map({0: 'Female', 1: 'Male'})

df['ResAge'] = df['RESPAge'][df['RESPRelationHHH']==100]
df.ResAge.fillna(df.HHHAge, inplace=True)
df['HHHAge'] = df['ResAge']

HHHAgebins = [16, 36, 65, np.inf]
HHHAgenames = ['16 to 35', '36 to 64', '65 and Older']

df['HHHAgeC'] = pd.cut(df['HHHAge'], HHHAgebins, labels=HHHAgenames)

In [6]:
HHHead = [col for col in df.columns if col[:3] == 'HHH']
list(HHHead)

['HHHSex', 'HHHAge', 'HHHGender', 'HHHAgeC']

In [7]:
HHSize = [col for col in df.columns if col[:6] == 'HHSize' and col[-1:] == 'F' or col[:6] == 'HHSize' and col[-1:] == 'M']
df['HHSize'] = df[HHSize].sum(axis=1)

HHSizebins = [0, 4, 8, 12, np.inf]
HHSizenames = ['1 to 4', '5 to 8', '9 to 12', '13 and More']

df['HHSizeC'] = pd.cut(df['HHSize'], HHSizebins, labels=HHSizenames)

In [8]:
Depnt = [col for col in df.columns if col[:6] == 'HHSize' and col[-5:] != '1864F' and col[-5:] != '1864M']
Depnt.remove('HHSize')
df['Depnt'] = df[Depnt].sum(axis=1)

In [9]:
Child = [col for col in df.columns if col[:6] == 'HHSize' and col[-5:] != '1864F' and col[-5:] != '1864M' and col[-6:] != 'AboveM' and col[-6:] != 'AboveF']
Child.remove('HHSize')
df['Child'] = df[Child].sum(axis=1)

In [10]:
Elderly = [col for col in df.columns if col[-6:] == 'AboveM' or col[-6:] == 'AboveF']

df['Elderly'] = df[Elderly].sum(axis=1)

In [11]:
df["NAdult"] = (df['HHSize'] - df['Depnt'])

In [12]:
df["DepRatio"] = df["Depnt"] / df["NAdult"]

DepRbins = [0, 1.5, 3, np.inf]
DepRnames = ['Low', 'High', 'Very High']

df['DepRatioC'] = pd.cut(df['DepRatio'], DepRbins, labels=DepRnames)

In [13]:
df['HHMainIncome'] = df['HHIncFirst_SRi'].replace({1:'Waged labor', 2:'Waged labor', 
                                                               3:'Waged labor', 4:'Waged labor', 5:'Other', 6:'Other',
                                                               7:'Other', 8:'Other', 9:'Other', 10:'Other', 999:'Other',
                                                               11:'Trade', 12:'Trade', 13:'Trade', 14:'Agriculture', 15:'Agriculture'})


# Food Security Indicators

## RCSi

In [14]:
df["RCSi"] = (df['rCSILessQlty']*1) + (df['rCSIBorrow']*2) + (df['rCSIMealSize']*1) + (df['rCSIMealAdult']*3) + (df['rCSIMealNb']*1)

## Food Consumption Score

In [15]:
df["FCS"] = (df['FCSStap']*2) + (df['FCSPulse']*3) + (df['FCSDairy']*4) + (df['FCSPr']*4) + (df['FCSVeg']*1) + (df['FCSFruit']*1) + (df['FCSFat']*.5) + (df['FCSSugar']*.5)

In [16]:
FCSbins = [0, 21, 35, np.inf]
FCSnames = ['Poor', 'Borderline', 'Acceptable']

df['FCSC'] = pd.cut(df['FCS'], FCSbins, labels=FCSnames)

## Food Consumtion Score 4 Point

In [17]:
df['FCS_4pt'] = pd.cut(df['FCS'], bins=[0, 21, 35, df['FCS'].max()], labels=[4, 3, 1])
df['FCS_4pt'] = np.where(df['RCSi'] >= 4, np.where(df['FCS_4pt'] == 1, 2, df['FCS_4pt']), df['FCS_4pt'])

## FCS Nutrition 

In [18]:
df["FGVitA"] = (df['FCSDairy']) + (df['FCSPrMeatO']) + (df['FCSPrEgg']) + (df['FCSVegOrg']) + (df['FCSVegGre']) + (df['FCSFruitOrg'])
df["FGProtein"] = (df['FCSPulse']) + (df['FCSDairy']) + (df['FCSPrMeatF']) + (df['FCSPrMeatO']) + (df['FCSPrFish']) + (df['FCSPrEgg'])
df["FGIron"] = (df['FCSPrMeatF']) + (df['FCSPrMeatO']) + (df['FCSPrFish'])

In [19]:
FCSNbins = [-np.inf, 0, 6, 42]
FCSNnames = ['Never consumed', 'Consumed sometimes', 'Consumed at least 7 times']

df['FGVitA'] = pd.cut(df['FGVitA'], FCSNbins, labels=FCSNnames)

In [20]:
FCSNbins = [-np.inf, 0, 6, 42]
FCSNnames = ['Never consumed', 'Consumed sometimes', 'Consumed at least 7 times']

df['FGProtein'] = pd.cut(df['FGProtein'], FCSNbins, labels=FCSNnames)

In [21]:
FCSNbins = [-np.inf, 0, 6, 42]
FCSNnames = ['Never consumed', 'Consumed sometimes', 'Consumed at least 7 times']

df['FGIron'] = pd.cut(df['FGIron'], FCSNbins, labels=FCSNnames)

## Dietary Diversity Score

In [22]:
df["HDDSVegSum"] = df['HDDSVegOrg'] + df['HDDSVegGre'] + df['HDDSVegOth']
df["HDDSPrMeatSum"] = df['HDDSPrMeatF'] + df['HDDSPrMeatO']
df["HDDSFruitSum"] = df['HDDSFruitOrg'] + df['HDDSFruitOth']

df["HDDSVeg"] = np.where(df['HDDSVegSum'] > 0, 1, df['HDDSVegSum'])
df["HDDSPrMeat"] = np.where(df['HDDSPrMeatSum'] > 0, 1, df['HDDSPrMeatSum'])
df["HDDSFruit"] = np.where(df['HDDSFruitSum'] > 0, 1, df['HDDSFruitSum'])

In [23]:
df['HDDs'] = df['HDDSStapCer'] + df['HDDSStapRoot'] + df['HDDSVeg'] + df['HDDSFruit'] + df['HDDSPrMeat'] + df['HDDSPrEgg'] + df['HDDSPrFish'] + df['HDDSPulse'] + df['HDDSDairy'] + df['HDDSFat']+ df['HDDSSugar'] + df['HDDSCond']

## DDS Categories

In [24]:
bins = [0, 2, 4, 6, np.inf]
names = ['2 and Less', '3 to 4', '5', '6 and more']

df['HDDsC'] = pd.cut(df['HDDs'], bins, labels=names)

# Livelihood Coping Strategies

In [25]:
LCSiIndList = ['Lcs_stress_DomAsset', 'Lcs_stress_Saving', 'Lcs_stress_EatOut', 'Lcs_stress_CrdtFood', 'Lcs_stress_BorrowCash', 'Lcs_stress_Pawn', 'Lcs_stress_LessSchool', 'Lcs_crisis_ProdAssets', 'Lcs_crisis_Health', 'Lcs_crisis_Edu', 'Lcs_stress_Edu', 'Lcs_crisis_Housing', 'Lcs_stress_Housing', 'Lcs_crisis_HHSeparation', 'Lcs_stress_HHSeparation', 'Lcs_crisis_OutSchool', 'Lcs_em_OutSchool', 'Lcs_crisis_Migration', 'Lcs_em_Migration', 'Lcs_crisis_ChildWork', 'Lcs_em_ChildWork', 'Lcs_em_Marriage', 'Lcs_crisis_Marriage', 'Lcs_em_ResAsset', 'Lcs_em_Begged', 'Lcs_em_IllegalAct', 'Lcs_em_ChildMigration', 'Lcs_stress_ConsActive', 'Lcs_crisis_ConsActive', 'Lcs_stress_SellFoodRation', 'Lcs_stress_SellNFIRation', 'Lcs_crisis_Utilities', 'Lcs_stress_Utilities', 'Lcs_crisis_Barter', 'Lcs_em_WildFood']

In [26]:
vars_to_process = [var for var in LCSiIndList if var in df.columns]
for var in vars_to_process:
    df[var] = df[var].replace({30:1, 20:1, 10:0, 9999:0})

In [27]:
stress_vars = [var for var in vars_to_process if 'stress' in var]
if stress_vars:
    df['stress_coping'] = df[stress_vars].sum(axis=1)
    df['stress_coping'] = np.where(df['stress_coping'] >= 1, 2, 0)

crisis_vars = [var for var in vars_to_process if 'crisis' in var]
if crisis_vars:
    df['crisis_coping'] = df[crisis_vars].sum(axis=1)
    df['crisis_coping'] = np.where(df['crisis_coping'] >= 1, 3, 0)

emergency_vars = [var for var in vars_to_process if '_em_' in var]
if emergency_vars:
    df['emergency_coping'] = df[emergency_vars].sum(axis=1)
    df['emergency_coping'] = np.where(df['emergency_coping'] >= 1, 4, 0)

In [28]:
df["LCSi"] = df[["stress_coping", "crisis_coping", "emergency_coping"]].max(axis=1)

# Household Food Expenditure Share

## Food Items Expenditure

In [29]:
HHFExpMN = [col for col in df.columns if col[:6] == 'HHExpF' and col[-5:] == 'MN_1M' and col[-15:] != 'HHExpFood_MN_1M']
df['HHExp_Food_MN_1M'] = df[HHFExpMN].sum(axis=1)

In [30]:
HHFExpCRD = [col for col in df.columns if col[:6] == 'HHExpF' and col[-6:] == 'CRD_1M' and col[-16:] != 'HHExpFood_CRD_1M']
df['HHExp_Food_CRD_1M'] = df[HHFExpCRD].sum(axis=1)

In [31]:
HHFExpOWN = [col for col in df.columns if col[:6] == 'HHExpF' and col[-6:] == 'Own_1M' and col[-16:] != 'HHExpFood_Own_1M']
df['HHExp_Food_Own_1M'] = df[HHFExpOWN].sum(axis=1)

In [32]:
HHFExpGift = [col for col in df.columns if col[:6] == 'HHExpF' and col[-10:] == 'GiftAid_1M' and col[-20:] != 'HHExpFood_GiftAid_1M']
df['HHExp_Food_GiftAid_1M'] = df[HHFExpGift].sum(axis=1)

### HH Monthly Food Expenditure

In [33]:
df["HHExpFood_1M"] = (df['HHExp_Food_MN_1M'] + df['HHExp_Food_CRD_1M'] + df['HHExp_Food_Own_1M'] + df['HHExp_Food_GiftAid_1M'])

## Food Expenditure W/Out Cond, Foodout & Bevs.

In [34]:
HHFExpMN2 = [col for col in HHFExpMN if (col[:6] == 'HHExpF' and col[-10:] != 'Cond_MN_1M' and col[-14:] != 'Beverage_MN_1M'
                                         and col[-9:] != 'Out_MN_1M')]
df['HHExp_Food_MN_1M2'] = df[HHFExpMN2].sum(axis=1)

In [35]:
HHFExpCRD2 = [col for col in HHFExpCRD if (col[:6] == 'HHExpF' and col[-11:] != 'Cond_CRD_1M' and col[-15:] != 'Beverage_CRD_1M'
                                         and col[-10:] != 'Out_CRD_1M')]
df['HHExp_Food_CRD_1M2'] = df[HHFExpCRD2].sum(axis=1)

In [36]:
HHFExpOWN2 = [col for col in HHFExpOWN if (col[:6] == 'HHExpF' and col[-11:] != 'Cond_OWN_1M' and col[-15:] != 'Beverage_OWN_1M'
                                         and col[-10:] != 'Out_OWN_1M')]
df['HHExp_Food_OWN_1M2'] = df[HHFExpOWN2].sum(axis=1)

In [37]:
HHFExpGift2 = [col for col in HHFExpOWN if (col[:6] == 'HHExpF' and col[-12:] != 'Cond_Gift_1M' and col[-16:] != 'Beverage_Gift_1M'
                                         and col[-11:] != 'Out_Gift_1M')]
df['HHExp_Food_GiftAid_1M2'] = df[HHFExpGift2].sum(axis=1)

In [38]:
df["HHExpFood_1M2"] = (df['HHExp_Food_MN_1M2'] + df['HHExp_Food_CRD_1M2'] + df['HHExp_Food_OWN_1M2'] + df['HHExp_Food_GiftAid_1M2'])

## Non-Food Item Expenditures

### NFI Cash Expenditure

In [39]:
HHExpNFINT_MN = [col for col in df.columns if (col[:7] == 'HHExpNF' and col[-10:] == 'Rent_MN_6M' or col[-13:] == 'MedServ_MN_6M'
                                          or col[-13:] == 'MedGood_MN_6M' or col[-11:] == 'Cloth_MN_6M'
                                          or col[-12:] == 'EduFee_MN_6M' or col[-13:] == 'EduGood_MN_6M'
                                          or col[-10:] == 'Soft_MN_6M' or col[-10:] == 'Sav_Est_6M'
                                          or col[-15:] == 'Insurance_MN_6M' or col[-11:] == 'Debt_Est_6M')]
df['HHExpNFTotal_MN_Int_30D'] = (df[HHExpNFINT_MN].sum(axis=1)/6)

In [40]:
HHExpNFCONS_MN = [col for col in df.columns if (col[:7] == 'HHExpNF' and col[-14:] == 'AlcTobac_MN_6M' or col[-9:] == 'Hyg_MN_6M'
                                          or col[-12:] == 'Transp_MN_6M' or col[-9:] == 'Wat_MN_6M'
                                          or col[-14:] == 'DwelServ_MN_6M' or col[-10:] == 'Elec_MN_6M'
                                          or col[-11:] == 'Energ_MN_6M' or col[-11:] == 'Phone_MN_6M')]
df['HHExpNFTotal_MN_CONS_30D'] = (df[HHExpNFCONS_MN].sum(axis=1)/6)

In [41]:
df["HHExpNFTotal_MN_1M"] = df['HHExpNFTotal_MN_Int_30D'] + df['HHExpNFTotal_MN_CONS_30D']

### NFI Credit Expenditure

In [42]:
HHExpNFInt_CRD = [col for col in df.columns if (col[:7] == 'HHExpNF' and col[-11:] == 'Rent_CRD_6M' or col[-14:] == 'MedServ_CRD_6M'
                                           or col[-14:] == 'MedGood_CRD_6M' or col[-12:] == 'Cloth_CRD_6M' 
                                            or col[-13:] == 'EduFee_CRD_6M' or col[-14:] == 'EduGood_CRD_6M'
                                           or col[-11:] == 'Soft_CRD_6M' or col[-16:] == 'Insurance_CRD_6M')]
df["HHExpNFTotal_CRD_Int_30D"] = (df[HHExpNFInt_CRD].sum(axis=1)/6)

In [43]:
HHExpNFCONS_CRD = [col for col in df.columns if (col[:7] == 'HHExpNF' and col[-15:] == 'AlcTobac_CRD_6M' or col[-10:] == 'Hyg_CRD_6M'
                                          or col[-13:] == 'Transp_CRD_6M' or col[-10:] == 'Wat_CRD_6M'
                                          or col[-15:] == 'DwelServ_CRD_6M' or col[-11:] == 'Elec_CRD_6M'
                                          or col[-12:] == 'Energ_CRD_6M' or col[-12:] == 'Phone_CRD_6M')]
df["HHExpNFTotal_CRD_CONS_30D"] = (df[HHExpNFCONS_CRD].sum(axis=1)/6)

In [44]:
df["HHExpNFTotal_CRD_1M"] = df['HHExpNFTotal_CRD_Int_30D'] + df['HHExpNFTotal_CRD_CONS_30D']

### NFI GiftAid Expenditure

In [45]:
HHExpNFInt_GiftAid = [col for col in df.columns if (col[:7] == 'HHExpNF' and col[-15:] == 'Rent_GiftAid_6M' or col[-18:] == 'MedServ_GiftAid_6M'
                                           or col[-18:] == 'MedGood_GiftAid_6M' or col[-16:] == 'Cloth_GiftAid_6M' 
                                            or col[-17:] == 'EduFee_GiftAid_6M' or col[-18:] == 'EduGood_GiftAid_6M'
                                           or col[-15:] == 'Soft_GiftAid_6M' or col[-20:] == 'Insurance_GiftAid_6M')]
df["HHExpNFTotal_GiftAid_Int_30D"] = (df[HHExpNFInt_GiftAid].sum(axis=1)/6)

In [46]:
HHExpNFCONS_GiftAid = [col for col in df.columns if (col[:7] == 'HHExpNF' and col[-19:] == 'AlcTobac_GiftAid_6M' or col[-14:] == 'Hyg_GiftAid_6M'
                                          or col[-17:] == 'Transp_GiftAid_6M' or col[-14:] == 'Wat_GiftAid_6M'
                                          or col[-19:] == 'DwelServ_GiftAid_6M' or col[-15:] == 'Elec_GiftAid_6M'
                                          or col[-16:] == 'Energ_GiftAid_6M' or col[-16:] == 'Phone_GiftAid_6M')]
df["HHExpNFTotal_GiftAid_CONS_30D"] = (df[HHExpNFCONS_GiftAid].sum(axis=1)/6)

In [47]:
df["HHExpNFTotal_GiftAid_1M"] = df['HHExpNFTotal_GiftAid_Int_30D'] + df['HHExpNFTotal_GiftAid_CONS_30D']

## Total Monthly NFI Household Expenditure

In [48]:
df["HHExpNFTotal_1M"] = df['HHExpNFTotal_MN_1M'] + df['HHExpNFTotal_CRD_1M'] + df['HHExpNFTotal_GiftAid_1M']

## Food Expenditure Share

In [49]:
df["HHExpTotal"] = (df['HHExpFood_1M'] + df['HHExpNFTotal_1M'])

In [50]:
df["FES"] = df['HHExpFood_1M'] / df["HHExpTotal"]

In [51]:
df["FES2"] = df['HHExpFood_1M2'] / (df['HHExpFood_1M2'] + df['HHExpNFTotal_1M'])

### FES Categories

In [52]:
bins = [0, 0.5, 0.65, 0.75, np.inf]
names = ['<50%', '50% to 65%', '65% to 75%', '>75%']

df['FESc'] = pd.cut(df['FES'], bins, labels=names)

### FES CARI Categories

In [53]:
bins = [-np.inf, 0.5, 0.65, 0.75, np.inf]
names = [1, 2, 3, 4]

df['FESCARI'] = pd.cut(df['FES'], bins, labels=names)

In [54]:
df.dropna(subset=["FESCARI"], inplace=True)

# ECMEN

In [55]:
df['ECMENs'] = df['HHExpTotal'] / (df['HHSize']*897.5) 

## ECMEN Categories

In [56]:
bins = [0, .74, 1, np.inf]
names = ['BelowfMEB', 'BelowMEB', 'AboveMEB']

df['ECMEN'] = pd.cut(df['ECMENs'], bins, labels=names)

## GAP Analysis & Transfer Value

In [83]:
df = df[df.HHExpTotal <= 100000]
df = df[df.HHSize >= 1]

In [90]:
df['Needs_GAP'] = ((df['HHSize']*897.5) - df['HHExpTotal'])

In [91]:
df["TransV"] = (df['Needs_GAP']/df['HHSize'])

In [92]:
df["TransV"].mean()

585.6065397157994

In [93]:
df.TransV.describe()

count    5081.000000
mean      585.606540
std       355.609779
min     -9726.296296
25%       545.192308
50%       659.895833
75%       736.388889
max       897.092593
Name: TransV, dtype: float64

# CARI WITH FES

In [57]:
df["LCSi"] = df["LCSi"].astype(int)
df["FESCARI"] = df["FESCARI"].astype(int)
df["FCS_4pt"] = df["FCS_4pt"].astype(int)

df['Mean_coping_capacity_FES'] = np.mean([df['LCSi'], df['FESCARI']], axis=0)
df['Mean_coping_capacity_FES'] = np.floor(df['Mean_coping_capacity_FES'])
df['CARI_unrounded_FES'] = np.mean([df['FCS_4pt'], df['Mean_coping_capacity_FES']], axis=0)
df['CARI_FES'] = np.ceil(df['CARI_unrounded_FES'])

In [58]:
df['CARI_FESc'] = np.where(df['CARI_FES'] == 1, 'Food Secure', df['CARI_FES'])
df['CARI_FESc'] = np.where(df['CARI_FES'] == 2, 'Marginally Food Secure', df['CARI_FESc'])
df['CARI_FESc'] = np.where(df['CARI_FES'] == 3, 'Moderately Food Insecure', df['CARI_FESc'])
df['CARI_FESc'] = np.where(df['CARI_FES'] == 4, 'Severely Food Insecure', df['CARI_FESc'])

### Results to Excel

# Table Outputs

### Demographics

In [59]:
var_list = ['HHHGender','HHHAgeC','HHSizeC', 'DepRatioC', 'HHMainIncome']
Indi_list = ['ADMIN0Name', 'ADMIN2Name']
if not os.path.exists('Results'):
    os.makedirs('Results')

for var in var_list:
    for ind in Indi_list:
    
        var_grouped = df.groupby(ind)
        var_counts = var_grouped[var].value_counts()
        var_percents = var_grouped[var].value_counts(normalize=True)
        var_results = pd.DataFrame({'Counts': var_counts, 'Percentage': var_percents})
        var_results = var_results.reset_index()
        var_results = pd.pivot_table(var_results, values='Percentage', index=[ind], columns=[var_results.columns[1]], aggfunc=np.sum)
        output_file = f"Results/{var}_{ind}.xlsx"    
        writer = pd.ExcelWriter(output_file, engine='xlsxwriter') 
        var_results.to_excel(writer, sheet_name='Results')
        writer.close()

### Food Security and Valnurebility Indicators

In [60]:
var_list = ['FCSC', 'FCS_4pt', 'HDDsC', 'RCSi','FGVitA','FGProtein','FGIron','LCSi', 'FESc', 'ECMEN', 'CARI_FESc']
Indi_list = ['ADMIN0Name', 'District', 'HHHGender', 'HHHAgeC', 'HHSizeC', 'DepRatioC', 'HHMainIncome']
if not os.path.exists('Results'):
    os.makedirs('Results')

for var in var_list:
    for ind in Indi_list:
    
        var_grouped = df.groupby(ind)
        var_counts = var_grouped[var].value_counts()
        var_percents = var_grouped[var].value_counts(normalize=True)
        var_results = pd.DataFrame({'Counts': var_counts, 'Percentage': var_percents})
        var_results = var_results.reset_index()
        var_results = pd.pivot_table(var_results, values='Percentage', index=[ind], columns=[var_results.columns[1]], aggfunc=np.sum)
        output_file = f"Results/{var}_{ind}.xlsx"    
        writer = pd.ExcelWriter(output_file, engine='xlsxwriter') 
        var_results.to_excel(writer, sheet_name='Results')
        writer.close()

### RCSi and HHSize Averages

In [61]:
var_list = ['RCSi','HHSize']
Indi_list = ['ADMIN0Name', 'District', 'HHHGender', 'HHHAgeC', 'HHSizeC', 'DepRatioC', 'HHMainIncome']
if not os.path.exists('Results'):
    os.makedirs('Results')

for var in var_list:
    for ind in Indi_list:
    
        var_grouped = df.groupby(ind)
        var_means = var_grouped[var].mean()
        var_results = pd.DataFrame({'Mean Value': var_means})
        var_results = var_results.reset_index()
        output_file = f"Results/{var}mean_{ind}.xlsx"    
        writer = pd.ExcelWriter(output_file, engine='xlsxwriter') 
        var_results.to_excel(writer, sheet_name='Results')
        writer.close()